<a href="https://colab.research.google.com/github/aish1012222/nyc_feb23/blob/main/humanfaces.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install mediapipe==0.10.11 mysql-connector-python --quiet


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.6/35.6 MB 43.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.9/33.9 MB 20.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.1/162.1 kB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 100.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 79.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 48.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [3]:
!pip install keras-facenet --quiet

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 39.7 MB/s eta 0:00:00


In [4]:
import os
import re
import json
import cv2
import time
import zipfile
import numpy as np
import mysql.connector
import tensorflow as tf
from tensorflow.keras import backend as K, optimizers
from tensorflow.keras.models import Model
from tensorflow.keras.layers import (
    Input, Conv2D, BatchNormalization, MaxPool2D,
    GlobalAveragePooling2D, Dense, Lambda, Concatenate
)
from sklearn.metrics.pairwise import cosine_similarity
from google.colab import drive
from keras_facenet import FaceNet

In [5]:
# ── 1) Mount Drive & Unzip Dataset ──
drive.mount('/content/drive')
ZIP_PATH   = '/content/drive/MyDrive/humanfaces1.zip'
EXTRACT_TO = '/content/humanfaces1'
with zipfile.ZipFile(ZIP_PATH, 'r') as z:
    z.extractall(EXTRACT_TO)
print('Dataset extracted to', EXTRACT_TO)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Dataset extracted to /content/humanfaces1


In [6]:
!apt-get update -qq
!apt-get install -qq mariadb-server


W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Preconfiguring packages ...
Selecting previously unselected package gawk.
(Reading database ... 126308 files and directories currently installed.)
Preparing to unpack .../0-gawk_1%3a5.1.0-1ubuntu0.1_amd64.deb ...
Unpacking gawk (1:5.1.0-1ubuntu0.1) ...
Selecting previously unselected package mariadb-common.
Preparing to unpack .../1-mariadb-common_1%3a10.6.22-0ubuntu0.22.04.1_all.deb ...
Unpacking mariadb-common (1:10.6.22-0ubuntu0.22.04.1) ...
Selecting previously unselected package galera-4.
Preparing to unpack .../2-galera-4_26.4.9-1build1_amd64.deb ...
Unpacking galera-4 (26.4.9-1build1) ...
Selecting previously unselected package libdbi-perl:amd64.
Preparing to unpack .../3-libdbi-perl_1.643-3build3_amd64.deb ...
Unpacking libdbi-perl:amd64 (1.643-3build3) ...
Selecting previously unselected pac

In [7]:
!mkdir -p /var/run/mysqld
!chown mysql:mysql /var/run/mysqld
!mysqld_safe --datadir='/var/lib/mysql' &


250625 22:37:14 mysqld_safe Logging to syslog.
250625 22:37:14 mysqld_safe Starting mariadbd daemon with databases from /var/lib/mysql


In [8]:
import time
time.sleep(10)  # wait to ensure server is ready

# Set root password and create database
!mysql -u root -e "ALTER USER 'root'@'localhost' IDENTIFIED BY 'Vaishnavi@1234'; FLUSH PRIVILEGES;"
!mysql -u root -pVaishnavi@1234 -e "CREATE DATABASE IF NOT EXISTS Demo1;"


In [9]:
ROOT_PW='Vaishnavi@1234'
db = mysql.connector.connect(
    host='localhost', user='root', password=ROOT_PW, database='Demo1'
)
cursor = db.cursor()
cursor.execute('''
CREATE TABLE IF NOT EXISTS FaceEmbeddings (
    id INT AUTO_INCREMENT PRIMARY KEY,
    label VARCHAR(100),
    image_name VARCHAR(100),
    embedding JSON
);
''')
db.commit()


In [10]:
import os, re, random
from collections import defaultdict

# ─── 1) Base extract path (update if your data lives elsewhere) ──────────────
EXTRACT_TO = '/content/humanfaces1/humanfaces'

# ─── 1) Generate label files in LABEL_DIR ─────────────────────────────────
IMAGE_DIR = os.path.join(EXTRACT_TO, 'Humans')   # adjust if needed
LABEL_DIR = os.path.join(EXTRACT_TO, 'labels')
os.makedirs(LABEL_DIR, exist_ok=True)

pattern = re.compile(r"(\d+)\s*\((\d+)\)\s*(\.\w+)", re.IGNORECASE)

image_files = sorted(f for f in os.listdir(IMAGE_DIR)
                     if f.lower().endswith(('.jpg','.jpeg','.png')))

for idx, img in enumerate(image_files, start=1):
    m = pattern.match(img)
    if m:
        # e.g. "1 (3707).jpg" → groups: prefix, number, .ext
        _, num, ext = m.groups()
        ext = ext.upper().lstrip('.')
        label_name = f"{num}.{ext}.txt"
    else:
        # fallback: "Divya.jpg" → "Divya.txt"
        label_name = os.path.splitext(img)[0] + '.txt'

    with open(os.path.join(LABEL_DIR, label_name), 'w') as f:
        f.write(str(idx))
print(f"✅ Generated {len(image_files)} label files in {LABEL_DIR!r}")

✅ Generated 7223 label files in '/content/humanfaces1/humanfaces/labels'


In [11]:
import os
import re
import random
from collections import defaultdict

# ─── 1) Point at your images directory ────────────────────────────────────────
IMAGE_DIR = '/content/humanfaces1/humanfaces/Humans'   # update to your actual path

# ─── 2) Build image → “true label” map (number in () or filename) ────────────
pattern = re.compile(r"\((\d+)\\)")
labels_map = {}

for img in os.listdir(IMAGE_DIR):
    if not img.lower().endswith(('.jpg', '.jpeg', '.png')):
        continue
    m = pattern.search(img)
    if m:
        label = m.group(1)                   # e.g. “3707”
    else:
        label = os.path.splitext(img)[0]     # e.g. “Divya”
    labels_map[img] = label

# ─── 3) Group filenames by label ─────────────────────────────────────────────
images_by_label = defaultdict(list)
for img, lbl in labels_map.items():
    images_by_label[lbl].append(os.path.join(IMAGE_DIR, img)) # Store full paths

# ─── 4) Split data into training and testing sets ───────────────────────────
train_images_by_label = {}
test_images_by_label = {}
test_size = 0.2 # Percentage of data to use for testing

all_labels = list(images_by_label.keys())

for lbl in all_labels:
    all_images = images_by_label[lbl]
    random.shuffle(all_images)
    split_idx = int(len(all_images) * (1 - test_size))
    train_images_by_label[lbl] = all_images[:split_idx]
    test_images_by_label[lbl] = all_images[split_idx:]

# ─── 5) Generate anchor–positive–negative triplets from training data ───────
triplets = []
train_labels = list(train_images_by_label.keys())

for lbl in train_labels:
    group = train_images_by_label[lbl]
    if len(group) < 2:
        continue                            # need ≥2 for anchor+positive
    for anchor in group:
        positive = random.choice([x for x in group if x != anchor])
        # Ensure negative is from a different label AND has at least one image
        neg_lbl = random.choice([l for l in train_labels if l != lbl and len(train_images_by_label[l]) > 0])
        negative = random.choice(train_images_by_label[neg_lbl])
        triplets.append((
            anchor,
            positive,
            negative,
        ))

random.shuffle(triplets)

print(f"✅ Total training triplets created: {len(triplets)}")
# Optional: preview the first 5 triplets
for a,p,n in triplets[:5]:
    print("A:", a, "P:", p, "N:", n)

# Split into lists
paths_a = [t[0] for t in triplets]
paths_p = [t[1] for t in triplets]
paths_n = [t[2] for t in triplets]


✅ Total triplets created: 379
A: /content/humanfaces1/humanfaces/Humans/1 (138).png P: /content/humanfaces1/humanfaces/Humans/1 (138).jpg N: /content/humanfaces1/humanfaces/Humans/1 (3829).jpg
A: /content/humanfaces1/humanfaces/Humans/1 (115).jpg P: /content/humanfaces1/humanfaces/Humans/1 (115).png N: /content/humanfaces1/humanfaces/Humans/1 (6807).jpg
A: /content/humanfaces1/humanfaces/Humans/1 (80).jpg P: /content/humanfaces1/humanfaces/Humans/1 (80).png N: /content/humanfaces1/humanfaces/Humans/1 (6292).jpg
A: /content/humanfaces1/humanfaces/Humans/1 (113).jpg P: /content/humanfaces1/humanfaces/Humans/1 (113).png N: /content/humanfaces1/humanfaces/Humans/1 (382).jpg
A: /content/humanfaces1/humanfaces/Humans/1 (33).jpeg P: /content/humanfaces1/humanfaces/Humans/1 (33).jpg N: /content/humanfaces1/humanfaces/Humans/1 (6211).jpg


In [13]:
IMG_SIZE    = 160
BATCH_SIZE  = 32
MARGIN      = 0.2
EPOCHS      = 10

def decode_image(path):
    img = tf.io.read_file(path)
    img = tf.image.decode_jpeg(img, channels=3)
    img = tf.image.resize(img, (IMG_SIZE, IMG_SIZE)) / 255.0
    return img

def load_triplet(a, p, n):
    return decode_image(a), decode_image(p), decode_image(n)

def format_triplet(a, p, n):
    # return a tuple of three inputs + dummy label
    return ({"anchor":a, "positive":p, "negative":n}, tf.zeros(()))

dataset = tf.data.Dataset.from_tensor_slices((paths_a, paths_p, paths_n))
train_ds = (
    dataset
    .shuffle(len(paths_a))
    .map(load_triplet,    num_parallel_calls=tf.data.AUTOTUNE)
    .map(format_triplet,  num_parallel_calls=tf.data.AUTOTUNE)
    .batch(BATCH_SIZE)
    .prefetch(tf.data.AUTOTUNE)
)

In [14]:
FACENET_H5  = '/content/drive/MyDrive/FaceNet_Project/facenet_keras.h5'

In [15]:
# point this to where your weights file actually lives

FACENET_H5  = '/content/drive/MyDrive/FaceNet_Project/facenet_keras.h5'
# ─── build your Facenet‐style base model ──────────────────────────────────────
def make_facenet_embedding():
    inp = Input((IMG_SIZE,IMG_SIZE,3), name="facenet_input")
    x = Conv2D(64, 7, strides=2, padding='same', activation='relu', name='conv1_conv')(inp)
    x = BatchNormalization(name='bn_conv1')(x)
    x = MaxPool2D(3,2,'same', name='pool1')(x)
    x = Conv2D(128,3,1,'same', activation='relu', name='conv2_conv')(x)
    x = BatchNormalization(name='bn_conv2')(x)
    x = MaxPool2D(3,2,'same', name='pool2')(x)
    x = GlobalAveragePooling2D(name='gap')(x)
    out = Dense(128, name='embeddings')(x)
    return Model(inp, out, name='facenet_base')

# ─── load pretrained weights ─────────────────────────────────────────────────
base = make_facenet_embedding()
base.load_weights(FACENET_H5, by_name=True, skip_mismatch=True)

# ─── extract just the embeddings layer for downstream use ────────────────────
embed_model = Model(base.input, base.get_layer('embeddings').output)

In [16]:
ia = Input((IMG_SIZE,IMG_SIZE,3), name='anchor')
ip = Input((IMG_SIZE,IMG_SIZE,3), name='positive')
in_ = Input((IMG_SIZE,IMG_SIZE,3), name='negative')

# Embeddings
ea = embed_model(ia)
ep = embed_model(ip)
en = embed_model(in_)

# L2‐normalize
norm = lambda x: K.l2_normalize(x, axis=1)
na = Lambda(norm, name='norm_anchor')(ea)
np_ = Lambda(norm, name='norm_positive')(ep)
nn = Lambda(norm, name='norm_negative')(en)

# Concatenate into one vector
merged = Concatenate(name='triplet_output')([na, np_, nn])

triplet_net = Model(inputs=[ia,ip,in_], outputs=merged)

In [17]:
def triplet_loss(y_true, y_pred):
    # y_pred shape = (batch, 128*3)
    a, p, n = tf.split(y_pred, num_or_size_splits=3, axis=1)
    pos_dist = tf.reduce_sum(tf.square(a - p), axis=1)
    neg_dist = tf.reduce_sum(tf.square(a - n), axis=1)
    return tf.reduce_mean(tf.maximum(pos_dist - neg_dist + MARGIN, 0.0))

triplet_net.compile(
    optimizer=optimizers.Adam(1e-4),
    loss=triplet_loss
)

triplet_net.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ anchor (InputLayer) │ (None, 160, 160,  │          0 │ -                 │
│                     │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ positive            │ (None, 160, 160,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ negative            │ (None, 160, 160,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ functional          │ (None, 128)       │    100,608 │ anchor[0][0],     │
│ (Functional)        │                   │            │ positive[0][0],   │
│                     │                   │            │ negative[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ norm_anchor         │ (None, 128)       │          0 │ functional[0][0]  │
│ (Lambda)            │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ norm_positive       │ (None, 128)       │          0 │ functional[1][0]  │
│ (Lambda)            │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ norm_negative       │ (None, 128)       │          0 │ functional[2][0]  │
│ (Lambda)            │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ triplet_output      │ (None, 384)       │          0 │ norm_anchor[0][0… │
│ (Concatenate)       │                   │            │ norm_positive[0]… │
│                     │                   │            │ norm_negative[0]… │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 100,608 (393.00 KB)

 Trainable params: 100,224 (391.50 KB)

 Non-trainable params: 384 (1.50 KB)

In [18]:
triplet_net.fit(train_ds, epochs=EPOCHS)

Epoch 1/10
12/12 ━━━━━━━━━━━━━━━━━━━━ 109s 8s/step - loss: 0.3329
Epoch 2/10
12/12 ━━━━━━━━━━━━━━━━━━━━ 93s 7s/step - loss: 0.2114
Epoch 3/10
12/12 ━━━━━━━━━━━━━━━━━━━━ 141s 7s/step - loss: 0.1857
Epoch 4/10
12/12 ━━━━━━━━━━━━━━━━━━━━ 144s 7s/step - loss: 0.1808
Epoch 5/10
12/12 ━━━━━━━━━━━━━━━━━━━━ 91s 7s/step - loss: 0.1771
Epoch 6/10
12/12 ━━━━━━━━━━━━━━━━━━━━ 145s 7s/step - loss: 0.1690
Epoch 7/10
12/12 ━━━━━━━━━━━━━━━━━━━━ 140s 7s/step - loss: 0.1706
Epoch 8/10
12/12 ━━━━━━━━━━━━━━━━━━━━ 142s 7s/step - loss: 0.1597
Epoch 9/10
12/12 ━━━━━━━━━━━━━━━━━━━━ 92s 7s/step - loss: 0.1639
Epoch 10/10
12/12 ━━━━━━━━━━━━━━━━━━━━ 159s 9s/step - loss: 0.1450


In [19]:
norm_anchor  = triplet_net.get_layer('norm_anchor').output
anchor_input = triplet_net.input[0]

# define lightweight model: face-img → 128-D L2-normalized embedding
ft_model = Model(inputs=anchor_input, outputs=norm_anchor, name='facenet_anchor_only')

# helper to load & preprocess exactly as during training:
def get_embedding(img_path):
    img = tf.io.read_file(img_path)
    img = tf.image.decode_jpeg(img, channels=3)
    img = tf.image.resize(img, (IMG_SIZE, IMG_SIZE)) / 255.0
    img = tf.expand_dims(img, axis=0)
    emb = ft_model.predict(img)
    return emb[0]  # shape (128,)

✅ Generated 7223 label files in '/content/humanfaces1/humanfaces/labels'


In [ ]:
import os, re, json

# same pattern you used before:
pattern = re.compile(r"(\d+)\s*\((\d+)\)\s*(\.\w+)", re.IGNORECASE)

IMAGE_DIR = os.path.join(EXTRACT_TO, 'Humans')
LABEL_DIR = os.path.join(EXTRACT_TO, 'labels')

inserted = 0
for img in sorted(os.listdir(IMAGE_DIR)):
    if not img.lower().endswith(('.jpg','.jpeg','.png')):
        continue

    # 1) recompute the label‐filename exactly as before
    m = pattern.match(img)
    if m:
        _, num, ext = m.groups()
        ext = ext.upper().lstrip('.')
        label_filename = f"{num}.{ext}.txt"
    else:
        label_filename = os.path.splitext(img)[0] + '.txt'

    # 2) read the numeric label
    label_path = os.path.join(LABEL_DIR, label_filename)
    if not os.path.isfile(label_path):
        print(f"⚠️ Missing {label_filename} for image {img}, skipping.")
        continue
    with open(label_path, 'r') as f:
        label_idx = f.read().strip()

    # 3) compute embedding
    img_path = os.path.join(IMAGE_DIR, img)
    emb = get_embedding(img_path)

    # 4) insert into MySQL
    cursor.execute(
        "INSERT INTO FaceEmbeddings (label, image_name, embedding) VALUES (%s, %s, %s)",
        (label_idx, img, json.dumps(emb.tolist()))
    )
    inserted += 1

db.commit()
print(f"✅ Inserted {inserted} embeddings.")


Streaming output truncated to the last 5000 lines.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 75ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 71ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step

In [22]:
cursor.execute("SELECT label, embedding FROM FaceEmbeddings")
rows = cursor.fetchall()
bank_labels = []
bank_embs   = []
for lab, emb_json in rows:
    bank_labels.append(lab)
    bank_embs.append(np.array(json.loads(emb_json)))
bank_embs = np.vstack(bank_embs)  # shape: (N,128)

In [23]:
def recognize(query_img_path, threshold=0.7):
    """
    Returns (label, similarity) if best match ≥ threshold, else (None, best_similarity).
    """
    q_emb = get_embedding(query_img_path).reshape(1, -1)
    sims  = cosine_similarity(q_emb, bank_embs).flatten()
    best_idx = np.argmax(sims)
    best_sim = float(sims[best_idx])
    if best_sim < threshold:
        return None, best_sim
    return bank_labels[best_idx], best_sim

In [24]:
test_path = '/content/humanfaces1/humanfaces/Humans/1 (55).jpeg'
label, sim = recognize(test_path, threshold=0.75)
if label:
    print(f"✅ Recognized as {label} (sim={sim:.3f})")
else:
    print(f"❌ No match (best sim={sim:.3f})")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 97ms/step
✅ Recognized as 2205 (sim=1.000)


In [40]:
!ls -lh /content/facenet_keras.h5


-rw-r--r-- 1 root root 89M Jun 25 19:27 /content/facenet_keras.h5


In [41]:
import h5py

path = '/content/facenet_keras.h5'
with h5py.File(path, 'r') as f:
    print("Top-level groups/datasets:", list(f.keys()))
    # e.g. ['conv1_conv', 'embeddings', 'dense_1', 'optimizer_weights', ...]


Top-level groups/datasets: ['model_weights']


TypeError: in user code:


    TypeError: outer_factory.<locals>.inner_factory.<locals>.tf__tf_preprocess() missing 1 required positional argument: 'n'


In [27]:
import random

# Suppose you have a held‐out `labels_map_test: Dict[label, List[paths]]`
# (disjoint from your train labels_map)

# Create N positive pairs (same label) and N negative pairs (different labels)
N = 1000
pos_pairs, neg_pairs = [], []
labels = list(labels_map_test.keys())

for _ in range(N):
    lbl = random.choice(labels)
    imgs = random.sample(labels_map_test[lbl], 2)
    pos_pairs.append((imgs[0], imgs[1]))

for _ in range(N):
    l1, l2 = random.sample(labels, 2)
    neg_pairs.append((random.choice(labels_map_test[l1]),
                      random.choice(labels_map_test[l2])))


NameError: name 'labels_map_test' is not defined

In [ ]:
from sklearn.metrics import accuracy_score

def pair_distance(a_path, b_path):
    e1, e2 = get_embedding(a_path), get_embedding(b_path)
    return np.linalg.norm(e1 - e2)

# Collect distances + true labels
dists, y_true = [], []
for a,b in pos_pairs:
    dists.append(pair_distance(a,b)); y_true.append(1)
for a,b in neg_pairs:
    dists.append(pair_distance(a,b)); y_true.append(0)


In [ ]:
# You can sweep thresholds to find the best operating point:
ths = np.linspace(0.5, 1.5, 50)
best_acc, best_th = 0, None
for th in ths:
    y_pred = [1 if d < th else 0 for d in dists]
    acc = accuracy_score(y_true, y_pred)
    if acc > best_acc:
        best_acc, best_th = acc, th

print(f"Best verification accuracy: {best_acc:.3f} at threshold {best_th:.3f}")


In [ ]:
from sklearn.metrics import roc_auc_score
auc = roc_auc_score(y_true, [-d for d in dists])  # negative dist = score
print(f"Verification ROC AUC: {auc:.3f}")
